In [5]:
import sqlite3
import re

sql_file = 'country_club.sql'  # Change this to your filename
db_file = 'country_club.db'
# 1. Read the SQL file
with open(sql_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# 2. Filter out MySQL-specific commands
cleaned_sql = []
for line in lines:
    # Skip lines starting with MySQL-only commands
    if line.strip().upper().startswith(('SET', 'START TRANSACTION', 'COMMIT', 'ROLLBACK', '/*')):
        continue
    
    # Clean up line content
    new_line = line.replace('`', '"')  # Replace backticks
    new_line = re.sub(r'ENGINE=.*;', ';', new_line, flags=re.IGNORECASE)
    new_line = re.sub(r'AUTO_INCREMENT=\d+', '', new_line, flags=re.IGNORECASE)
    
    cleaned_sql.append(new_line)

# 3. Create the DB and execute
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

try:
    # Join the lines back together and execute
    cursor.executescript("".join(cleaned_sql))
    conn.commit()
    print("Success! The .db file is ready.")
    
    # Quick check: List tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    print("Tables found:", [t[0] for t in cursor.fetchall()])
    
except sqlite3.Error as e:
    print(f"An error occurred: {e}")
finally:
    conn.close()


Success! The .db file is ready.
Tables found: ['Bookings', 'Facilities', 'Members']


In [19]:
#* Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members! */


import sqlite3
import pandas as pd

# 1. Connect to your local SQLite database
conn = sqlite3.connect('country_club.db')

# 2. Define the Q13 SQL query
# We use strftime('%m', ...) to extract the month from the starttime string
query = """
SELECT name, SUM(revenue) AS total_revenue
FROM (
    SELECT f.name,
           CASE WHEN b.memid = 0 THEN b.slots * f.guestcost
                ELSE b.slots * f.membercost END AS revenue
    FROM Bookings AS b
    JOIN Facilities AS f ON b.facid = f.facid
) AS rev_table
GROUP BY name
HAVING total_revenue < 1000
ORDER BY total_revenue;
"""

# 3. Execute the query and load results into a pandas DataFrame
df_usage = pd.read_sql_query(query, conn)

# 4. Display the results
print(df_usage)

# 5. Close the connection
conn.close()



            name  total_revenue
0   Table Tennis            180
1  Snooker Table            240
2     Pool Table            270


In [23]:
#/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


import sqlite3
import pandas as pd

# 1. Connect to your local SQLite database
conn = sqlite3.connect('country_club.db')

# 2. Define the Q13 SQL query
# We use strftime('%m', ...) to extract the month from the starttime string
query = """
SELECT m1.surname || ', ' || m1.firstname AS member,
       m2.surname || ', ' || m2.firstname AS recommended_by
FROM Members AS m1
LEFT JOIN Members AS m2 ON m1.recommendedby = m2.memid
WHERE m1.memid > 0
ORDER BY m2.surname, m2.firstname;
"""

# 3. Execute the query and load results into a pandas DataFrame
df_usage = pd.read_sql_query(query, conn)

# 4. Display the results
print(df_usage)

# 5. Close the connection
conn.close()







                      member      recommended_by
0              Smith, Darren                None
1               Smith, Tracy                None
2                Rownam, Tim                None
3              Tracy, Burton                None
4            Farrell, Jemima                None
5             Farrell, David                None
6       Tupperware, Hyacinth                None
7              Smith, Darren                None
8          Sarwin, Ramnaresh     Bader, Florence
9               Coplin, Joan      Baker, Timothy
10          Genting, Matthew     Butters, Gerald
11            Baker, Timothy     Farrell, Jemima
12             Pinker, David     Farrell, Jemima
13         Rumney, Henrietta    Genting, Matthew
14            Jones, Douglas        Jones, David
15               Dare, Nancy    Joplette, Janice
16              Jones, David    Joplette, Janice
17                Hunt, John  Purview, Millicent
18               Boothe, Tim         Rownam, Tim
19          Joplette

In [21]:
#/* Q12: Find the facilities with their usage by member, but not guests */


import sqlite3
import pandas as pd

# 1. Connect to your local SQLite database
conn = sqlite3.connect('country_club.db')

# 2. Define the Q13 SQL query
# We use strftime('%m', ...) to extract the month from the starttime string
query = """
SELECT f.name, COUNT(b.bookid) AS usage_count
FROM Facilities AS f
JOIN Bookings AS b ON f.facid = b.facid
WHERE b.memid != 0
GROUP BY f.name;
"""

# 3. Execute the query and load results into a pandas DataFrame
df_usage = pd.read_sql_query(query, conn)

# 4. Display the results
print(df_usage)

# 5. Close the connection
conn.close()




              name  usage_count
0  Badminton Court          344
1   Massage Room 1          421
2   Massage Room 2           27
3       Pool Table          783
4    Snooker Table          421
5     Squash Court          195
6     Table Tennis          385
7   Tennis Court 1          308
8   Tennis Court 2          276


In [13]:
#/* Q13: Find the facilities usage by month, but not guests */

import sqlite3
import pandas as pd

# 1. Connect to your local SQLite database
conn = sqlite3.connect('country_club.db')

# 2. Define the Q13 SQL query
# We use strftime('%m', ...) to extract the month from the starttime string
query = """
SELECT 
    f.name AS facility_name, 
    strftime('%m', b.starttime) AS month, 
    COUNT(b.bookid) AS usage_count
FROM Facilities AS f
JOIN Bookings AS b ON f.facid = b.facid
WHERE b.memid != 0
GROUP BY f.name, month
ORDER BY month, usage_count DESC;
"""

# 3. Execute the query and load results into a pandas DataFrame
df_usage = pd.read_sql_query(query, conn)

# 4. Display the results
print(df_usage)

# 5. Close the connection
conn.close()

      facility_name month  usage_count
0        Pool Table    07          103
1    Massage Room 1    07           77
2     Snooker Table    07           68
3    Tennis Court 1    07           65
4   Badminton Court    07           51
5      Table Tennis    07           48
6    Tennis Court 2    07           41
7      Squash Court    07           23
8    Massage Room 2    07            4
9        Pool Table    08          272
10    Snooker Table    08          154
11   Massage Room 1    08          153
12     Table Tennis    08          143
13  Badminton Court    08          132
14   Tennis Court 1    08          111
15   Tennis Court 2    08          109
16     Squash Court    08           85
17   Massage Room 2    08            9
18       Pool Table    09          408
19    Snooker Table    09          199
20     Table Tennis    09          194
21   Massage Room 1    09          191
22  Badminton Court    09          161
23   Tennis Court 1    09          132
24   Tennis Court 2    09

In [15]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('country_club.db')

# Define the Q13 query
# We use strftime to group by month and filter out guests (memid 0)
query = """
SELECT 
    f.name AS facility_name, 
    strftime('%m', b.starttime) AS month, 
    COUNT(b.bookid) AS usage_count
FROM Facilities AS f
JOIN Bookings AS b ON f.facid = b.facid
WHERE b.memid != 0
GROUP BY f.name, month
ORDER BY month, usage_count DESC;
"""

# Execute and store in a DataFrame
df_q13 = pd.read_sql_query(query, conn)

# Close connection and display the result
conn.close()
df_q13

,facility_name,month,usage_count
0,Pool Table,07,103
1,Massage Room 1,07,77
2,Snooker Table,07,68
3,Tennis Court 1,07,65
4,Badminton Court,07,51
5,Table Tennis,07,48
6,Tennis Court 2,07,41
7,Squash Court,07,23
8,Massage Room 2,07,4
9,Pool Table,08,272
